In [1]:
# imports

# standard
from IPython.display import display, Markdown

# langchain
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.embeddings import GPT4AllEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

# langchain-ollama
from langchain_ollama import ChatOllama


/Users/cmoyacal/miniforge3/envs/langchain/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in GPT4AllEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
# read doc
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [3]:
docs = loader.load()

In [4]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [5]:
# define embeddings for doc and query
embeddings = GPT4AllEmbeddings()

Downloading: 100%|██████████| 45.9M/45.9M [00:01<00:00, 26.8MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 469MiB/s]


In [6]:
# compute sample embedding vectors
embed = embeddings.embed_query("Hi my name is Harrison")

In [7]:
print(len(embed))

384


In [8]:
print(embed[:5])

[-0.06153647601604462, -0.06207933649420738, -0.019020892679691315, 0.04829559102654457, -0.02856493555009365]


In [9]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

/Users/cmoyacal/miniforge3/envs/langchain/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [10]:
# define query for docs
query = "Please suggest a shirt with sunblocking"

In [11]:
# check similarity
docs = db.similarity_search(query)

In [12]:
len(docs)

4

In [13]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.')

In [14]:
# define retriever
retriever = db.as_retriever()

In [15]:
# define llm
llm = ChatOllama(
    model="llama3.1",
    temperature=0.0,
)

In [16]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [17]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 

/var/folders/z5/s8fr1m057mj7x5qy6x65x9qw0000gn/T/ipykernel_53297/720143539.py:1: LangChainDeprecationWarning: The method `BaseChatModel.call_as_llm` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.call_as_llm(f"{qdocs} Question: Please list all your \


In [18]:
display(Markdown(response))

Here are the shirts with sun protection listed in a table:

| **Shirt Name** | **Description** | **Key Features** |
| --- | --- | --- |
| Sun Shield Shirt by [Brand] | High-performance sun shirt for blocking UV rays | UPF 50+, moisture-wicking, quick-drying, abrasion-resistant |
| Tropical Breeze Shirt | Lightweight, breathable long-sleeve men's UPF shirt | UPF 50+, wrinkle-resistant, moisture-wicking, front and back cape venting |
| Men's Plaid Tropic Shirt, Short-Sleeve | Ultracomfortable sun protection for fishing and travel | UPF 50+, wrinkle-free, quick-drying, front and back cape venting, two front bellows pockets |
| Women's Tropical Tee, Sleeveless | Five-star sleeveless button-up shirt with SunSmart protection | UPF 50+, wrinkle-resistant, low-profile pockets, side shaping, front and back cape venting |

Here's a brief summary of each shirt:

1. **Sun Shield Shirt**: A high-performance sun shirt designed for blocking UV rays, perfect for outdoor activities.
2. **Tropical Breeze Shirt**: A lightweight, breathable long-sleeve men's UPF shirt ideal for hot weather, fishing, and travel.
3. **Men's Plaid Tropic Shirt**: A short-sleeve shirt with ultracomfortable sun protection, designed for fishing and travel.
4. **Women's Tropical Tee**: A sleeveless button-up shirt with SunSmart protection, perfect for warm weather activities.

All of these shirts offer UPF 50+ protection, which is the highest rated sun protection possible, blocking 98% of the sun's harmful UV rays.

In [19]:
# define qNa  
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [20]:
# define query
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [21]:
# obtain response from qa retrieval
response = qa_stuff.run(query)

/var/folders/z5/s8fr1m057mj7x5qy6x65x9qw0000gn/T/ipykernel_53297/187079000.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_stuff.run(query)




> Entering new RetrievalQA chain...

> Finished chain.


In [22]:
# display response from retrieval
display(Markdown(response))

Here are the shirts with sun protection:

| **Shirt Name** | **Description** |
| --- | --- |
| Women's Tropical Tee, Sleeveless | A sleeveless button-up shirt with built-in UPF 50+ rated fabric for superior sun protection. Softly shapes the body and has a flattering fit. |
| Sun Shield Shirt by [Brand] | A high-performance sun shirt that blocks 98% of harmful UV rays. Wicks moisture for quick-drying comfort and is abrasion resistant. |
| Sunrise Tee | A lightweight, breathable button-down shirt with built-in UPF 50+ rated fabric for superior sun protection. Wicks away moisture and dries quickly. |
| Tropical Breeze Shirt (Men's) | A long-sleeve men's shirt with superior SunSmart protection from the sun's harmful rays. Wrinkle-resistant and moisture-wicking fabric keeps you cool and comfortable. |

Let me know if you'd like me to add anything else!

In [ ]:
# you can do the same thing with index